# Lab 6 - Multiple Linear Regression

In [5]:
import pandas as pd
import numpy as np 
from plotnine import ggplot, aes, geom_bar, stat_qq, stat_qq_line, position_dodge, geom_col, theme_minimal
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms

## Question Set 1

Use the same cars2010 dataset you used in lab 4. This dataset has variables pertaining to fuel economy of various cars. Do not create a training and test dataset; just use the whole cars2010 dataset for analysis. The cars2011 and cars2012 datasets will be used at later time periods. 

In [6]:
cars = pd.read_csv("https://raw.githubusercontent.com/IAA-Faculty/statistical_foundations/refs/heads/master/cars2010.csv")

### Question 1a

Run a regression predicting the FE variable using all the remaining variable. Some of these predictor variables are coded as numeric, but should be treated as categorical. The only numeric predictor should be EngDispl. All remaining predictors are categorical. What is the F-statistic from the global F-test? Round to 2 decimals.

In [7]:
cars.columns

Index(['EngDispl', 'NumCyl', 'Transmission', 'FE', 'AirAspirationMethod',
       'NumGears', 'TransLockup', 'TransCreeperGear', 'DriveDesc',
       'IntakeValvePerCyl', 'ExhaustValvesPerCyl', 'CarlineClassDesc',
       'VarValveTiming', 'VarValveLift'],
      dtype='object')

In [8]:
fe_mlr = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(AirAspirationMethod) + C(NumGears) + C(TransLockup) + C(TransCreeperGear) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveTiming) + C(VarValveLift)", data = cars).fit()
fe_mlr.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     FE   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.825
Method:                 Least Squares   F-statistic:                     95.55
Date:                Sun, 20 Jul 2025   Prob (F-statistic):               0.00
Time:                        21:50:59   Log-Likelihood:                -2808.7
No. Observations:                1107   AIC:                             5729.
Df Residuals:                    1051   BIC:                             6010.
Df Model:                          55                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================
                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                 32.2203      2.492     12.930      0.000      27.331      37.110
C(NumCyl)[T.3]                                            10.6247      2.128      4.994      0.000       6.450      14.800
C(NumCyl)[T.4]                                             4.9337      0.766      6.443      0.000       3.431       6.436
C(NumCyl)[T.5]                                             2.0652      0.803      2.573      0.010       0.490       3.640
C(NumCyl)[T.6]                                             1.3619      0.619      2.202      0.028       0.148       2.576
C(NumCyl)[T.8]                                            -0.3181      0.651     -0.488      0.625      -1.596       0.960
C(NumCyl)[T.10]                                           -2.8488      1.021     -2.791      0.005      -4.852      -0.846
C(NumCyl)[T.12]                                           -3.8567      0.977     -3.948      0.000      -5.774      -1.940
C(NumCyl)[T.16]                                           -3.1459      3.153     -0.998      0.319      -9.333       3.041
C(Transmission)[T.A5]                                     -2.3615      0.597     -3.955      0.000      -3.533      -1.190
C(Transmission)[T.A6]                                      1.7328      0.439      3.949      0.000       0.872       2.594
C(Transmission)[T.A7]                                     12.2980      2.276      5.404      0.000       7.833      16.763
C(Transmission)[T.AM6]                                    -2.8658      0.855     -3.351      0.001      -4.544      -1.188
C(Transmission)[T.AM7]                                     7.1905      2.483      2.896      0.004       2.319      12.062
C(Transmission)[T.AV]                                      2.1907      1.253      1.749      0.081      -0.267       4.649
C(Transmission)[T.AVS6]                                   -0.1352      1.452     -0.093      0.926      -2.983       2.713
C(Transmission)[T.M5]                                     -2.8311      0.648     -4.371      0.000      -4.102      -1.560
C(Transmission)[T.M6]                                     -0.4170      0.423     -0.986      0.324      -1.247       0.413
C(Transmission)[T.Other]                                   6.5932      1.516      4.350      0.000       3.619       9.567
C(Transmission)[T.S4]                                     -3.4856      0.966     -3.606      0.000      -5.382      -1.589
C(Transmission)[T.S5]                                     -3.0088      0.653     -4.610      0.000      -4.290      -1.728
C(Transmission)[T.S6]                                      1.5232      0.378      4.033      0.000       0.782       2.264
C(Transm

Answer: 95.55

### Question 1b

What percentage of variability can be explained by this model? Round to 1 decimal (remember, the question asks for what percentage)

Answer: 83.3

### Question 1c

Is the average effect that EngDispl has on FE significant?

Answer: at p < 0.001, yes

### Question 1d

What is the correct interpretation of the EngDispl coefficient?

Answer: When holding all other predictors fixed, FE decreases, on average, by 2.25 for each 1 unit increase of EngDispl

### Question 1e

Trying to evaluate categorical variables in traditional linear regression output can be difficult because the p-values are for each categorical dummy variable as a whole; you need a global p-value for each categorical variable. Use the anova_lm() function on your linear regression object to get the p-values for each global categorical variable. Which of the variables has the highest p-value?

In [9]:
sms.anova_lm(fe_mlr, typ=2)

,sum_sq,df,F,PR(>F)
C(NumCyl),1174.983331,8.0,14.895615,8.367743e-21
C(Transmission),4236.726588,15.0,28.645467,5.332169e-68
C(AirAspirationMethod),151.227576,2.0,7.668629,4.938823e-04
C(NumGears),827.464969,5.0,16.784034,5.565166e-16
C(TransLockup),61.837772,1.0,6.271488,1.242019e-02
C(TransCreeperGear),43.436361,1.0,4.405246,3.606668e-02
C(DriveDesc),1535.045672,4.0,38.920467,2.054919e-30
C(IntakeValvePerCyl),206.275090,3.0,6.973363,1.200995e-04
C(ExhaustValvesPerCyl),133.263412,2.0,6.757681,1.213072e-03
C(CarlineClassDesc),3495.403655,16.0,22.156139,7.275457e-56


Answer: VarValveTiming

### Question 1f

Is the answer in part e) significant?

Answer: at p = 0.58, no

### Question 1g

Rerun the preceding model, but remove the variable from part e) (regardless of significance). Compare with the preceding model. Based on the global F-test, r-squared value, and adjusted r-squared value, is there a large difference between this model and the preceeding model?

In [10]:
fe_mlr_reduced = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(AirAspirationMethod) + C(NumGears) + C(TransLockup) + C(TransCreeperGear) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
fe_mlr_reduced.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     FE   R-squared:                       0.833
Model:                            OLS   Adj. R-squared:                  0.825
Method:                 Least Squares   F-statistic:                     97.38
Date:                Sun, 20 Jul 2025   Prob (F-statistic):               0.00
Time:                        21:50:59   Log-Likelihood:                -2808.9
No. Observations:                1107   AIC:                             5728.
Df Residuals:                    1052   BIC:                             6003.
Df Model:                          54                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================
                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                 32.2503      2.490     12.950      0.000      27.364      37.137
C(NumCyl)[T.3]                                            10.6080      2.127      4.988      0.000       6.435      14.781
C(NumCyl)[T.4]                                             4.9254      0.765      6.435      0.000       3.424       6.427
C(NumCyl)[T.5]                                             2.0461      0.802      2.553      0.011       0.473       3.619
C(NumCyl)[T.6]                                             1.3671      0.618      2.211      0.027       0.154       2.580
C(NumCyl)[T.8]                                            -0.2740      0.646     -0.424      0.672      -1.542       0.994
C(NumCyl)[T.10]                                           -2.7991      1.016     -2.754      0.006      -4.793      -0.805
C(NumCyl)[T.12]                                           -3.8080      0.973     -3.915      0.000      -5.716      -1.900
C(NumCyl)[T.16]                                           -3.0733      3.149     -0.976      0.329      -9.253       3.106
C(Transmission)[T.A5]                                     -2.3494      0.596     -3.939      0.000      -3.520      -1.179
C(Transmission)[T.A6]                                      1.7297      0.439      3.943      0.000       0.869       2.590
C(Transmission)[T.A7]                                     12.3182      2.275      5.416      0.000       7.855      16.781
C(Transmission)[T.AM6]                                    -2.8627      0.855     -3.349      0.001      -4.540      -1.185
C(Transmission)[T.AM7]                                     7.1950      2.482      2.899      0.004       2.325      12.065
C(Transmission)[T.AV]                                      2.2002      1.252      1.757      0.079      -0.257       4.657
C(Transmission)[T.AVS6]                                   -0.1260      1.451     -0.087      0.931      -2.973       2.721
C(Transmission)[T.M5]                                     -2.8307      0.647     -4.372      0.000      -4.101      -1.560
C(Transmission)[T.M6]                                     -0.4189      0.423     -0.991      0.322      -1.248       0.411
C(Transmission)[T.Other]                                   6.6026      1.515      4.358      0.000       3.629       9.576
C(Transmission)[T.S4]                                     -3.5228      0.964     -3.655      0.000      -5.414      -1.632
C(Transmission)[T.S5]                                     -3.0144      0.652     -4.620      0.000      -4.295      -1.734
C(Transmission)[T.S6]                                      1.5242      0.378      4.037      0.000       0.783       2.265
C(Transm

Answer: no

### Question 1h

Now, rerun the preceding model (from part g), but eliminate the variables with the highest p-value (determined using the same method from part e) and rerunning the regression until you only have variables significant at the 0.008 level. Remember to run the model after EACH variable you remove as the p-value might change by removing a variable.

Did the global F-statistic, r-squared, and adjusted r-squared values changed largely (hint for decision on F-stat: did it change the p-value much)?

In [11]:
sms.anova_lm(fe_mlr_reduced, typ=2)

,sum_sq,df,F,PR(>F)
C(NumCyl),1178.237356,8.0,14.946802,7.008112e-21
C(Transmission),4267.809313,15.0,28.874817,1.593418e-68
C(AirAspirationMethod),153.821248,2.0,7.805340,4.316072e-04
C(NumGears),825.994807,5.0,16.765357,5.794614e-16
C(TransLockup),61.743310,1.0,6.266072,1.245783e-02
C(TransCreeperGear),50.484022,1.0,5.123414,2.380820e-02
C(DriveDesc),1545.618918,4.0,39.214612,1.233751e-30
C(IntakeValvePerCyl),216.192343,3.0,7.313488,7.441133e-05
C(ExhaustValvesPerCyl),135.022153,2.0,6.851419,1.105806e-03
C(CarlineClassDesc),3504.204950,16.0,22.226701,4.765496e-56


In [12]:
fe_mlr_reduced_2 = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(AirAspirationMethod) + C(NumGears) + C(TransLockup) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
sms.anova_lm(fe_mlr_reduced_2, typ=2)

,sum_sq,df,F,PR(>F)
C(NumCyl),1209.867846,8.0,15.288192,2.179973e-21
C(Transmission),4267.258398,15.0,28.758475,2.832668e-68
C(AirAspirationMethod),143.082751,2.0,7.232117,7.594649e-04
C(NumGears),827.261786,5.0,16.725578,6.324970e-16
C(TransLockup),64.455078,1.0,6.515763,1.083241e-02
C(DriveDesc),1563.074747,4.0,39.502804,7.485878e-31
C(IntakeValvePerCyl),222.238359,3.0,7.488691,5.813586e-05
C(ExhaustValvesPerCyl),114.696210,2.0,5.797320,3.133406e-03
C(CarlineClassDesc),3478.612079,16.0,21.978305,1.954870e-55
C(VarValveLift),80.584105,1.0,8.146247,4.399629e-03


In [13]:
fe_mlr_reduced_3 = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(AirAspirationMethod) + C(NumGears) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
sms.anova_lm(fe_mlr_reduced_3, typ=2)

,sum_sq,df,F,PR(>F)
C(NumCyl),1224.399282,8.0,15.391269,1.529746e-21
C(Transmission),4203.155421,15.0,28.178998,5.548163e-67
C(AirAspirationMethod),129.023576,2.0,6.487546,1.583784e-03
C(NumGears),784.437366,5.0,15.777189,5.273529e-15
C(DriveDesc),1589.505864,4.0,39.961658,3.390264e-31
C(IntakeValvePerCyl),220.721257,3.0,7.398851,6.597101e-05
C(ExhaustValvesPerCyl),112.004950,2.0,5.631818,3.690705e-03
C(CarlineClassDesc),3537.836222,16.0,22.236125,4.357873e-56
C(VarValveLift),80.197335,1.0,8.064943,4.599777e-03
EngDispl,687.729499,1.0,69.160641,2.778337e-16


In [14]:
fe_mlr_reduced_4 = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(NumGears) + C(DriveDesc) + C(IntakeValvePerCyl) + C(ExhaustValvesPerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
sms.anova_lm(fe_mlr_reduced_4, typ=2)

,sum_sq,df,F,PR(>F)
C(NumCyl),1168.598557,8.0,14.538726,2.799652e-20
C(Transmission),4272.148787,15.0,28.346935,2.213700e-67
C(NumGears),766.410158,5.0,15.256055,1.691778e-14
C(DriveDesc),1839.253899,4.0,45.764918,1.735889e-35
C(IntakeValvePerCyl),206.982275,3.0,6.866934,1.394292e-04
C(ExhaustValvesPerCyl),80.675817,2.0,4.014804,1.832265e-02
C(CarlineClassDesc),3472.655936,16.0,21.601941,1.631345e-54
C(VarValveLift),127.190443,1.0,12.659177,3.903249e-04
EngDispl,634.191043,1.0,63.120597,4.957859e-15
Residual,10609.939901,1056.0,NaN,NaN


In [15]:
fe_mlr_reduced_5 = smf.ols("FE ~ EngDispl + C(NumCyl) + C(Transmission) + C(NumGears) + C(DriveDesc) + C(IntakeValvePerCyl) + C(CarlineClassDesc) + C(VarValveLift)", data = cars).fit()
sms.anova_lm(fe_mlr_reduced_5, typ=2)

,sum_sq,df,F,PR(>F)
C(NumCyl),1161.935686,8.0,14.457593,3.686724e-20
C(Transmission),4916.649325,15.0,32.627356,7.533389e-77
C(NumGears),847.701242,5.0,16.876280,4.492311e-16
C(DriveDesc),1831.600372,4.0,45.580033,2.357193e-35
C(IntakeValvePerCyl),518.714869,3.0,17.211208,6.451652e-11
C(CarlineClassDesc),3475.465798,16.0,21.622054,1.428113e-54
C(VarValveLift),125.977703,1.0,12.540001,4.156912e-04
EngDispl,655.485713,1.0,65.247989,1.789924e-15
Residual,10618.693503,1057.0,NaN,NaN


In [16]:
fe_mlr_reduced_5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                     FE   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     104.7
Date:                Sun, 20 Jul 2025   Prob (F-statistic):               0.00
Time:                        21:51:00   Log-Likelihood:                -2822.2
No. Observations:                1107   AIC:                             5744.
Df Residuals:                    1057   BIC:                             5995.
Df Model:                          49                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================
                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                 31.6742      2.519     12.574      0.000      26.731      36.617
C(NumCyl)[T.3]                                            11.1609      2.208      5.054      0.000       6.827      15.494
C(NumCyl)[T.4]                                             5.9422      0.898      6.620      0.000       4.181       7.704
C(NumCyl)[T.5]                                             3.0227      0.954      3.168      0.002       1.150       4.895
C(NumCyl)[T.6]                                             2.3837      0.809      2.948      0.003       0.797       3.970
C(NumCyl)[T.8]                                             0.6403      0.876      0.731      0.465      -1.079       2.360
C(NumCyl)[T.10]                                           -1.7033      1.216     -1.401      0.162      -4.089       0.683
C(NumCyl)[T.12]                                           -3.0351      1.162     -2.612      0.009      -5.315      -0.755
C(NumCyl)[T.16]                                           -2.7760      3.300     -0.841      0.400      -9.252       3.700
C(Transmission)[T.A5]                                     -2.7497      0.612     -4.494      0.000      -3.950      -1.549
C(Transmission)[T.A6]                                      1.6457      0.424      3.880      0.000       0.813       2.478
C(Transmission)[T.A7]                                     12.0581      2.282      5.284      0.000       7.581      16.536
C(Transmission)[T.AM6]                                    -2.9772      0.861     -3.459      0.001      -4.666      -1.289
C(Transmission)[T.AM7]                                     7.7111      2.502      3.082      0.002       2.802      12.620
C(Transmission)[T.AV]                                      2.5580      1.259      2.031      0.042       0.087       5.029
C(Transmission)[T.AVS6]                                    0.2949      1.459      0.202      0.840      -2.567       3.157
C(Transmission)[T.M5]                                     -2.2272      0.610     -3.654      0.000      -3.423      -1.031
C(Transmission)[T.M6]                                     -0.0545      0.385     -0.142      0.887      -0.810       0.701
C(Transmission)[T.Other]                                   7.0748      1.522      4.647      0.000       4.087      10.062
C(Transmission)[T.S4]                                     -3.8807      0.969     -4.003      0.000      -5.783      -1.979
C(Transmission)[T.S5]                                     -3.6087      0.665     -5.427      0.000      -4.913      -2.304
C(Transmission)[T.S6]                                      1.2294      0.373      3.294      0.001       0.497       1.962
C(Transm

Answer: No

## Question Set 2

Using the bike data from previous labs, you want to build a couple of different models and see which one is better. We will learn in later lectures how to do this with test datasets, but for now, we will only do this with trianing data. First, we need to split the data into training and test. Run the following code to get the training and test split (since the data is randomly divided, use the same seed or we will not end up with the same data, thus we will not necessarily get the same results).

seed = 123
np.random.seed(seed)

bike['id'] = bike.index + 1
train = bike.sample(frac=0.7, random_state=seed)
test = bike[~bike['id'].isin(train['id'])]

In [17]:

bike = pd.read_csv("https://raw.githubusercontent.com/IAA-Faculty/statistical_foundations/refs/heads/master/bike.csv")

seed = 123
np.random.seed(seed)

bike['id'] = bike.index + 1
train = bike.sample(frac=0.7, random_state=seed)
test = bike[~bike['id'].isin(train['id'])]

### Question 2a

How many observations are in the train dataset?

In [18]:
train.shape

(12165, 17)

### Question 2b

How many observations are in the test dataset?

In [19]:
test.shape

(5214, 17)

### Question 2c

Since you know that temp and atemp are highly correlated, you can't decide which variable is better to predict number of users (cnt). You probably shouldn't use both in your model since you will have multicollinearity issues, so one strategy is to build two models and compare them to see which predicts better. 

Using the training data, build one model with the following variables:
actual temperature (temp), humidity (hum), and wind speed (windspeed).

What is the adjusted r-squared values for this model? (Round to 3 decimals)

In [20]:
bike_model_1 = smf.ols("cnt ~ temp + hum + windspeed", data = bike).fit()
bike_model_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.251
Model:                            OLS   Adj. R-squared:                  0.251
Method:                 Least Squares   F-statistic:                     1945.
Date:                Sun, 20 Jul 2025   Prob (F-statistic):               0.00
Time:                        21:51:01   Log-Likelihood:            -1.1253e+05
No. Observations:               17379   AIC:                         2.251e+05
Df Residuals:                   17375   BIC:                         2.251e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    175.8100      6.187     28.416      0.000     163.683     187.937
temp         362.5344      6.205     58.427      0.000     350.372     374.697
hum         -273.4651      6.469    -42.270      0.000    -286.146    -260.784
windspeed     26.3198     10.180      2.585      0.010       6.366      46.274
==============================================================================
Omnibus:                     3606.701   Durbin-Watson:                   0.417
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7255.109
Skew:                           1.244   Prob(JB):                         0.00
Kurtosis:                       4.957   Cond. No.                         12.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Question 2d

Is the global F-test significant?

Answer: At p < 0.01, yes

### Question 2e

Which variables are significant at the .01 significance level?

Answer: At p < 0.001, both temp and humidity are. At p = 0.01, windspeed is arguably significant

### Question 2f

Build a second model wih the following variables: feeling temperature (atemp), humidity (hum), and wind speed (windspeed). What is the adjusted r-squared value? (round to 3 decimals)

In [21]:
bike_model_2 = smf.ols("cnt ~ atemp + hum + windspeed", data = bike).fit()
bike_model_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    cnt   R-squared:                       0.253
Model:                            OLS   Adj. R-squared:                  0.253
Method:                 Least Squares   F-statistic:                     1964.
Date:                Sun, 20 Jul 2025   Prob (F-statistic):               0.00
Time:                        21:51:01   Log-Likelihood:            -1.1250e+05
No. Observations:               17379   AIC:                         2.250e+05
Df Residuals:                   17375   BIC:                         2.250e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    158.6952      6.335     25.049      0.000     146.277     171.113
atemp        409.2251      6.952     58.864      0.000     395.598     422.852
hum         -275.8632      6.458    -42.716      0.000    -288.522    -263.205
windspeed     47.8602     10.191      4.697      0.000      27.886      67.835
==============================================================================
Omnibus:                     3588.025   Durbin-Watson:                   0.420
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7231.830
Skew:                           1.236   Prob(JB):                         0.00
Kurtosis:                       4.968   Cond. No.                         12.6
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### Question 2g

Is the global F-test significant?

Answer: At p < 0.01, yes

### Question 2h

Which variables are significant at the .01 significance level?

Answer: At p < 0.001, temp, humidity, and windspeed are all significant

### Question 2i

Ideally, we would compare these models using test data. We will learn how to do this in the future for labs. For now, which model has the higher adjusted r-squared in training data?

Answer: Model 2